In [9]:
import numpy as np
import xarray as xr

import os
from dolfin import *


from mmdisk.fea.fenics.utils import get_loadfunc, get_time_list_alt
from mmdisk.fea.common import create_time_points
from mmdisk.fea.fenics.mesh import mesh_and_properties, mesh2d_and_properties
from mmdisk.fea.fenics.incremental import incremental_cyclic_fea
from mmdisk.fea.common import default_cycle

import matplotlib.pyplot as plt

from mmdisk.fea.fenics.utils import (
    AbPETScContext,
    as_3D_tensor,
    get_boundary,
    ppos,
)

In [2]:
disks2d = xr.open_zarr("one_disk_with_full_fgm_comp_prop_1000_Cre_calib_rev_fix_chunk_1_for_fea.zarr")
mesh, props = mesh_and_properties(disks2d.isel(disks=0).load())
# mesh, props = mesh_and_properties(picked_disk.load())
omega = 1000
T_load = 200
t_heatdwell = 20.0
t_cooldwell = 1500.  # cooling dwell time
skip_cold = False  # skip the cold dwell for the thermal problem

l_cycle = default_cycle.copy()
l_cycle[1] = t_heatdwell
l_cycle[3] = t_cooldwell

time_division =  [0.25, 0.25, 0.25, 0.25]

time_steps = l_cycle.copy() * time_division

load = get_loadfunc(T_load, l_cycle)

n_cycles = 10

# t_fine = l_cycle[:3].sum() + l_cycle[2]
period, _, t_output = create_time_points(n_cycles, l_cycle, output_division=2.0)
t_list = get_time_list_alt(l_cycle, time_steps, n_cycles, skip_cold=skip_cold)

In [3]:
mesh = mesh
properties = props
load = T_load
omega = omega
t_list = t_list
period = period
t_output = t_output

In [4]:
rho, cp, kappa, E, sig0, nu, alpha_V = properties

rho

In [5]:
# material properties
rho, cp, kappa, E, sig0, nu, alpha_V = properties
lmbda = E * nu / (1 + nu) / (1 - 2 * nu)
mu = E / 2.0 / (1 + nu)
Et = E / 1e5  # tangent modulus - any better method for perfect plasticity?
H = E * Et / (E - Et)  # hardening modulus


In [6]:
# DEFINE THERMAL PROBLEM
V = FunctionSpace(mesh, "P", 1)  # Temperature space
v = TestFunction(V)
x = SpatialCoordinate(mesh)  # Coords
T_initial = Constant(0.0)
T_pre = interpolate(T_initial, V)  # Temp. at last time step
T_old = interpolate(T_initial, V)  # ** Temp. at last mechanical step **
T_crt = TrialFunction(V)
dt = Constant(1.0)
F_thermal = (rho * cp * (T_crt - T_pre) / dt) * v * x[0] * dx + kappa * dot(
    grad(T_crt), grad(v)
) * x[0] * dx
a_thermal, L_thermal = lhs(F_thermal), rhs(F_thermal)

# DEFINE MECH PROBLEM
U = VectorFunctionSpace(mesh, "CG", 1)
We = VectorElement(
    "Quadrature", mesh.ufl_cell(), degree=1, dim=4, quad_scheme="default"
)
W = FunctionSpace(mesh, We)
W0e = FiniteElement("Quadrature", mesh.ufl_cell(), degree=1, quad_scheme="default")
W0 = FunctionSpace(mesh, W0e)

metadata = {"quadrature_degree": 1, "quadrature_scheme": "default"}
dxm = dx(metadata=metadata)

sig = Function(W)
sig_old = Function(W)
n_elas = Function(W)
beta = Function(W0)
p = Function(W0, name="Cumulative plastic strain")
strain = Function(W, name="Strain vector")
u = Function(U, name="Total displacement")
du = Function(U, name="Iteration correction")
Du = Function(U, name="Current increment")
v_ = TrialFunction(U)
u_ = TestFunction(U)

In [7]:
def eps(v):
    return sym(
        as_tensor(
            [
                [v[0].dx(0), 0, v[0].dx(1)],
                [0, v[0] / x[0], 0],
                [v[1].dx(0), 0, v[1].dx(1)],
            ]
        )
    )

def eps_to_vector(v):
    """Convert strain tensor to vector format [eps_rr, eps_theta, eps_zz, eps_rz]"""
    eps_tensor = eps(v)
    return as_vector(
        [eps_tensor[0, 0], eps_tensor[1, 1], eps_tensor[2, 2], eps_tensor[0, 2]]
    )

def sigma(v, dT):
    return (lmbda * tr(eps(v)) - alpha_V * (3 * lmbda + 2 * mu) * dT) * Identity(
        3
    ) + 2.0 * mu * eps(v)

def proj_sig(v, dT, old_sig, old_p):
    sig_n = as_3D_tensor(old_sig)
    sig_elas = sig_n + sigma(v, dT)
    s = dev(sig_elas)
    sig_eq = sqrt(3 / 2.0 * inner(s, s))
    f_elas = sig_eq - sig0 - H * old_p
    dp = ppos(f_elas) / (3 * mu + H)
    n_elas = s / sig_eq * ppos(f_elas) / f_elas
    beta = 3 * mu * dp / sig_eq
    new_sig = sig_elas - beta * s
    return (
        as_vector([new_sig[0, 0], new_sig[1, 1], new_sig[2, 2], new_sig[0, 2]]),
        as_vector([n_elas[0, 0], n_elas[1, 1], n_elas[2, 2], n_elas[0, 2]]),
        beta,
        dp,
    )

def update_sig_thermal(dT, old_sig):
    sig_n = as_3D_tensor(old_sig)
    new_sig = sig_n - alpha_V * (3 * lmbda + 2 * mu) * dT * Identity(3)
    return as_vector([new_sig[0, 0], new_sig[1, 1], new_sig[2, 2], new_sig[0, 2]])

def sigma_tang(v):
    N_elas = as_3D_tensor(n_elas)
    e = eps(v)
    return (
        lmbda * tr(e) * Identity(3)
        + 2.0 * mu * e
        - 3 * mu * (3 * mu / (3 * mu + H) - beta) * inner(N_elas, e) * N_elas
        - 2 * mu * beta * dev(e)
    )

def local_project(v, V, u=None):
    dv = TrialFunction(V)
    v_ = TestFunction(V)
    a_proj = inner(dv, v_) * dxm
    b_proj = inner(v, v_) * dxm
    solver = LocalSolver(a_proj, b_proj)
    solver.factorize()
    if u is None:
        u = Function(V)
        solver.solve_local_rhs(u)
        return u
    else:
        solver.solve_local_rhs(u)
        return

In [10]:
omega_ = Constant(omega)
F_int = rho * omega_**2 * x[0] * u_[0]

a_Newton = inner(eps(v_), sigma_tang(u_)) * x[0] * dxm
res = -inner(eps(u_), as_3D_tensor(sig)) * x[0] * dxm + F_int * x[0] * dxm


In [11]:
u_

Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2)), 0, None)